In [132]:
import pandas as pd
from RAKE.rake import Rake
import json
import ast
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/omai.r/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
matched_dfs = pd.read_csv('data/intermediate/matched_dfs.csv', converters = {'clean_text': ast.literal_eval})

In [73]:
matched_dfs = pd.read_csv('data/intermediate/matched_dfs_merged.csv')

In [53]:
nature2references   = pd.read_hdf('data/nature/preprocessed/Nature2References.hdf')
nature_references_info = pd.read_hdf('data/nature/preprocessed/NatureReferencesInformation.hdf').set_index('DOI')

In [48]:
nature2references = pd.merge(nature2references, nature_references_info[['ArticleID']].reset_index(), right_on = 'ArticleID', left_on = 'CitingArticleID')

In [67]:
#matched_dfs = pd.merge(nature2references, matched_dfs,
#                       right_on = 'ArticleID', left_on = 'CitedArticleID')

matched_dfs = pd.merge(matched_dfs, nature_references_info[['ArticleID', 'Title']])

In [68]:
matched_dfs

,ArticleID,text,count,clean_text,Title
0,pub.1000000277,[['A detailed comparison of residues involved ...,1,[A detailed comparison of residues involved in...,Molecular Determinants of Agonist Discriminati...
1,pub.1000000280,"[['Also, the epigenetic stability of human ES ...",1,"[Also, the epigenetic stability of human ES ce...",Temporal and parental-specific expression of i...
2,pub.1000000588,[['Hospital and clinic HTC had similar costs (...,1,[Hospital and clinic HTC had similar costs (&d...,What Is the Cost of Providing Outpatient HIV C...
3,pub.1000000718,"[['<b>e</b>, Allele-based pleiotropy of <i>IL6...",2,"[e, Allele-based pleiotropy of IL6R 358Asp (rs...",Identification of multiple risk variants for a...
4,pub.1000000866,"[['However, structural<bibr rid=""b39 b40 b41 b...",1,"[However, structural and biochemical studies o...","Crystal Structure of an α1,4-N-Acetylhexosamin..."
...,...,...,...,...,...
304441,pub.1121819253,[['Lineage-affiliated transcription factors<bi...,4,[Lineage-affiliated transcription factors with...,GATA-1 interacts with the myeloid PU.1 transcr...
304442,pub.1121819356,[['\r\n<p>In order to investigate a yolk-sac c...,1,[In order to investigate a yolk-sac contributi...,VEGF-C signaling pathways through VEGFR-2 and ...
304443,pub.1121824901,[['Despite the clear upper limit in observed a...,1,[Despite the clear upper limit in observed axi...,Sea-Floor Tectonics and Submarine Hydrothermal...
304444,pub.1122267945,[['The so-called photon-dominated or photodiss...,7,[The so-called photon-dominated or photodissoc...,Physics of the Interstellar and Intergalactic ...


In [69]:
matched_dfs.head()

,ArticleID,text,count,clean_text,Title
0,pub.1000000277,[['A detailed comparison of residues involved ...,1,[A detailed comparison of residues involved in...,Molecular Determinants of Agonist Discriminati...
1,pub.1000000280,"[['Also, the epigenetic stability of human ES ...",1,"[Also, the epigenetic stability of human ES ce...",Temporal and parental-specific expression of i...
2,pub.1000000588,[['Hospital and clinic HTC had similar costs (...,1,[Hospital and clinic HTC had similar costs (&d...,What Is the Cost of Providing Outpatient HIV C...
3,pub.1000000718,"[['<b>e</b>, Allele-based pleiotropy of <i>IL6...",2,"[e, Allele-based pleiotropy of IL6R 358Asp (rs...",Identification of multiple risk variants for a...
4,pub.1000000866,"[['However, structural<bibr rid=""b39 b40 b41 b...",1,"[However, structural and biochemical studies o...","Crystal Structure of an α1,4-N-Acetylhexosamin..."


In [55]:
nature_references_info.head()

,ArticleID,PubYear,Title,Volume,Issue,Cites,Journal,NatureID
DOI,,,,,,,,
10.1038/166444b0,pub.1007899685,1950,Detection of Sugars on Paper Chromatograms,166,4219,3143,Nature,166444b0
10.1038/166117b0,pub.1026639700,1950,The Soil-Plate Method for Isolation of Fungi f...,166,4211,394,Nature,166117b0
10.1038/165514a0,pub.1039717355,1950,Reversible Dilation and Contraction by Changin...,165,4196,359,Nature,165514a0
10.1038/166780a0,pub.1011096023,1950,Cell Specificity of Histones,166,4227,280,Nature,166780a0
10.1038/166288a0,pub.1000041177,1950,Synthesis of Peptides in Enzymic Reactions inv...,166,4216,259,Nature,166288a0


In [58]:
matched_dfs.loc[4]['clean_text']

['e, Allele-based pleiotropy of IL6R 358Asp (rs2228145 (A)) on multiple disease phenotypes, including increased risk of RA, ankylosing spondylitis and coronary heart disease (asterisks indicate associations obtained from the literature) and protection from asthma, as well as levels of biomarkers (increased C-reactive protein (CRP) and fibrinogen but decreased soluble interleukin-6 receptor (sIL6R)).',
 'For ankylosing spondylitis, atopic dermatitis, primary sclerosing cholangitis, juvenile idiopathic arthritis, and psoriasis, Immunochip studies had been previously been published, but only the lead SNPs from associated Immunochip regions were available.']

In [23]:
rake = Rake("RAKE/SmartStoplist.txt")

### Test on a few papers

In [74]:
def top_trigram(text):
    #print(text)
    
    
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(' '.join(text))
    filtered_sentence = [w for w in word_tokens if (w not in stop_words) and (w.isalpha())]
    
    distribution = nltk.FreqDist(nltk.trigrams(filtered_sentence))
    distribution =  pd.DataFrame.from_dict(distribution, orient = 'index')
    distribution['freq'] = distribution[0] / distribution[0].sum()
    
    
    
    return distribution.sort_values('freq', ascending = False).iloc[:10]

In [35]:
matched_dfs.iloc[105089]['text']

'[[\'The density and distribution of stomata in the epidermis of aerial organs is modulated by intrinsic developmental programs, by hormones and by environmental factors such as light, humidity and carbon dioxide<bibr rid="b1 b2 b13 b14"/>.\']]'

In [24]:
top_trigram(matched_dfs.iloc[105089]['clean_text'])

['The density and distribution of stomata in the epidermis of aerial organs is modulated by intrinsic developmental programs, by hormones and by environmental factors such as light, humidity and carbon dioxide.']


,0,freq
"(environmental, factors, light)",1,0.0625
"(modulated, intrinsic, developmental)",1,0.0625
"(factors, light, humidity)",1,0.0625
"(stomata, epidermis, aerial)",1,0.0625
"(epidermis, aerial, organs)",1,0.0625
"(aerial, organs, modulated)",1,0.0625
"(organs, modulated, intrinsic)",1,0.0625
"(intrinsic, developmental, programs)",1,0.0625
"(developmental, programs, hormones)",1,0.0625
"(The, density, distribution)",1,0.0625


In [86]:
test_df = matched_dfs.sort_values('count', ascending = False).iloc[:300].sample(5)

In [87]:
for index, paper in test_df.iterrows():
    
    print('\n\n')
    print('Paper Title: %s \n\nPaper DOI: %s \n\n Paper Keywords: %s\n\n' % (paper['Title'],paper['DOI'],
                                                                          str(rake.run(paper['clean_text']))))
    




Paper Title: A Bivalent Chromatin Structure Marks Key Developmental Genes in Embryonic Stem Cells 

Paper Keywords: 10.1016/j.cell.2006.02.041 

 PaperID: [('primary human fibroblast cell types', 21.5), ('actively silence differentiation-affiliated transcription factors', 20.7), ('h3k27me pathway disrupt cardiac development', 19.3), ('chromatin regulatory networks controlling pluripotency', 18.77894736842105), ('bivalent domains mark developmentally important loci', 17.57016632016632), ('systematically analysed chromatin-modifying proteins', 15.0), ('non-expressed target genes adopts', 14.45945945945946), ('maps show close agreement', 13.5), ('mouse cage tag clusters', 13.4), ('previously reported chip-chip data', 13.0), ('properly restore bivalent domains', 12.793243243243243), ('human frontal cortex h3k4me3', 12.666666666666666), ('regulating key developmental genes', 12.316602316602317), ('poised h3k4me3+h3k27me3+ bivalent status', 11.925061425061426), ('mark active enhancers spe

In [91]:
test_df[test_df['DOI'] == '10.1016/j.cell.2006.02.041']['clean_text'].values

array(['[\'The 5′ element, for which we suggest the name HOTTIP for ‘HOXA transcript at the distal tip’, exhibits bivalent H3K4me3 and H3K27me3, a histone modification pattern associated with poised regulatory sequences.\', \'The combination of both activating (H3K4 methylation) and inactivating (H3K27 methylation) chromatin marks characterizes ‘poised’ promoters and enhancers, which regulate expression of key developmental genes.\', \'H3K4me is an activating mark found in promoters/enhancers of key developmental genes.\', \'These findings establish that mutations in many genes in the H3K4me–H3K27me pathway disrupt cardiac development and are consistent with previous evidence implicating these chromatin marks in regulating key developmental genes, including those involved in cardiac development.\', \'Lysine 27 is a critical residue of histone 3 and its variants, and methylation at this position (H3K27me), which may be mimicked by the terminal CH3 of methionine substituted at this resid

### Investigating Keywords for Scale Free

In [184]:
matched_dfs[matched_dfs['Title'].str.contains(r'Scaling.*Random.*')]\
            .sort_values('count', ascending = False)

,ArticleID,text,count,clean_text,Title
45845,pub.1010080128,[['Subjects were assigned a location in a netw...,31,[Subjects were assigned a location in a networ...,Emergence of Scaling in Random Networks


In [122]:
rake.run(matched_dfs.iloc[145265]['clean_text'])

[('basic laws governing human motion remains limited owing', 64.0),
 ('hierarchically organized community structures spanning inner-city', 36.0),
 ('escherichia coli k-12 mg1655 strain', 25.0),
 ('temporally varying sensory inputs', 16.0),
 ('find relevant link communities', 16.0),
 ('including major biological networks', 13.095238095238095),
 ('behavioural responses invoking', 9.0),
 ('maintaining pervasive overlap', 9.0),
 ('large social network', 9.0),
 ('large-scale social network', 9.0),
 ('small biological circuits', 8.666666666666666),
 ('mobile phone network', 8.5),
 ('metabolic network iaf1260', 8.5),
 ('control temporal networks', 7.428571428571429),
 ('protein interaction data', 7.166666666666666),
 ('protein interaction networks', 7.095238095238095),
 ('metabolic networks', 4.928571428571429),
 ('protein interaction', 4.666666666666666),
 ('mobile viruses', 4.5),
 ('data sets', 4.5),
 ('synchronized networks', 4.428571428571429),
 ('communication networks', 4.42857142857142

In [21]:
top_trigram(matched_dfs.iloc[145265]['clean_text'])

['These issues are particularly acute for neuroimaging given the high dimensionality of the data, relatively low statistical power of many studies, high degree of analytic flexibility in data analysis procedures, and potential for questionable research practices such as circular analysis procedures.']


,0,freq
"(analysis, procedures, potential)",1,0.037037
"(analytic, flexibility, data)",1,0.037037
"(many, studies, high)",1,0.037037
"(research, practices, circular)",1,0.037037
"(neuroimaging, given, high)",1,0.037037
"(particularly, acute, neuroimaging)",1,0.037037
"(issues, particularly, acute)",1,0.037037
"(degree, analytic, flexibility)",1,0.037037
"(relatively, low, statistical)",1,0.037037
"(high, degree, analytic)",1,0.037037


In [70]:
matched_dfs[matched_dfs['Title'].str.contains(r'.*Coot:.*')]\
            .sort_values('count', ascending = False)
#[['Title', 'DOI', 'ArticleID', 'count', 'Cites']]

,ArticleID,text,count,clean_text,Title
92208,pub.1020281098,[['This model was combined into a single prote...,520,[This model was combined into a single protein...,Coot: model-building tools for molecular graphics


In [71]:
matched_dfs.iloc[92208]

ArticleID                                        pub.1020281098
text          [['This model was combined into a single prote...
count                                                       520
clean_text    [This model was combined into a single protein...
Title         Coot: model-building tools for molecular graphics
Name: 92208, dtype: object

In [129]:
rake.run(matched_dfs.iloc[105089]['clean_text'])

[('experimentally phased ncs averaged electron density map permitted',
  32.00457459207459),
 ('protein data bank accession number 4r12', 26.90426948687818),
 ('protein data bank accession number 3mku', 26.90426948687818),
 ('protein data bank accession number 4hyc', 26.90426948687818),
 ('resulting maps showed clear unbiased density', 24.48301282051282),
 ('selenium anomalous difference fourier peak', 23.0),
 ('u2009å resolution sigmaa-weighted difference fourier maps',
  22.254901960784313),
 ('highly reliable initial rigid body fit', 21.57804459691252),
 ('iterative process involving careful model building', 20.778171702225674),
 ('multi-domain averaged electron density maps', 20.676923076923078),
 ('two-fold symmetry averaging produced experimental maps', 19.78181818181818),
 ('composite annealed omit maps calculated', 18.04285714285714),
 ('coot applying ten-fold ncs operations', 18.004032258064516),
 ('native eif3d data set processed', 16.927234927234927),
 ('pdb accession number

In [186]:
top_trigram(matched_dfs.iloc[45845]['clean_text'])


,0,freq
"(protein, interaction, networks)",2,0.003317
"(degree, distribution, Here)",2,0.003317
"(random, graphs, networks)",2,0.003317
"(networks, degree, distributions)",2,0.003317
"(highly, connected, proteins)",2,0.003317
"(networks, social, networks)",2,0.003317
"(popularity, similarity, instead)",1,0.001658
"(among, units, made)",1,0.001658
"(described, may, seem)",1,0.001658
"(links, existing, nodes)",1,0.001658


In [104]:
test_trigrams = top_trigram(matched_dfs.iloc[92208]['clean_text'])

### Build TF-IDF based key words system

In [7]:
#clean_text_corpus = [' '.join(ast.literal_eval(i)) for i in tqdm(matched_dfs['clean_text'].tolist())]
clean_text_corpus = [' '.join(i) for i in tqdm(matched_dfs['clean_text'].tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [8]:
clean_text_corpus[:50]

["\nMethods\n\n\nFossil fuel definitions\nA ‘McKelvey’ box is often used to provide an overview of the relationship between different resource and reserve estimates. Limb darkening was treated with a quadratic approximation, with coefficients initialized at values determined from model atmospheres. The primary data for our tomographic VS model consists of a large set of dispersion curves for broadband Rayleigh and Love wave group and phase velocities.  We aim to further develop DNCs to serve as representational engines for one-shot learning, scene understanding, language processing and cognitive mapping, capable of intuiting the variable structure and scale of the world within a single, generic model. Artemisinin, a sesquiterpene lactone endoperoxide extracted from Artemisia annua L (family Asteraceae; commonly known as sweet wormwood), is highly effective against multi-drug-resistant Plasmodium spp., but is in short supply and unaffordable to most malaria sufferers.  We anticipate tha

In [161]:
def tf_idfer(clean_text_corpus):

    print('Getting stop words.. ')
    stop_words = set(stopwords.words('english')) 

    print('Tokenizing the sentences..')
    clean_text_corpus = [word_tokenize(sent) for sent in clean_text_corpus]

    print('Cleaning the sentences..')
    cleaner = lambda sent: [w for w in sent if (w not in stop_words) and (w.isalpha())]
    clean_text_corpus = list(map(cleaner, clean_text_corpus))

    print('Joining the words back to make a sentence..')
    clean_text_corpus = [' '.join(i) for i in clean_text_corpus]

    print('Vectorizing the sentence..')
    vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 3))
    filtered_corpus_vector = vectorizer.fit_transform(clean_text_corpus)
    
    #print(len(vectorizer.get_feature_names()))
    #print(filtered_corpus_vector.toarray())
    
    return vectorizer, filtered_corpus_vector
    

In [162]:
%%time
vectorizer, filtered_corpus_vector = tf_idfer(clean_text_corpus)

Getting stop words.. 
Tokenizing the sentences..
Cleaning the sentences..
Joining the words back to make a sentence..
Vectorizing the sentence..
CPU times: user 6min 4s, sys: 4.53 s, total: 6min 9s
Wall time: 6min 8s


##### The one with no mapping has a lot of sentences. This should be removed

In [126]:
[len(i) for i in clean_text_corpus[:10]]

[92399, 242, 176, 130, 654, 229, 284, 183, 147, 146]

In [163]:
trigrams = vectorizer.get_feature_names()

In [164]:
transformer = TfidfTransformer(smooth_idf=True)

In [165]:
tf_idf = transformer.fit_transform(filtered_corpus_vector)

In [167]:
tf_idf = tf_idf[1:]
filtered_corpus_vector = filtered_corpus_vector[1:]

In [168]:
filtered_corpus_vector.shape

(304446, 8565201)

In [169]:
tf_idf.shape

(304446, 8565201)

In [108]:
[' '.join(list(i)) for i in top_trigram(matched_dfs.iloc[92208]['clean_text']).index.astype(list)]

['The model built',
 'manual model building',
 'model built using',
 'model building refinement',
 'building refinement performed',
 'initial model built',
 'Model building performed',
 'performed using COOT',
 'model building COOT',
 'model building carried']

In [129]:
len(matched_dfs)

304446

### Looking into the tf-idf for coot to see if it makes sense.

In [170]:
trigrams[np.argmax(tf_idf[92208])]

'using coot'

In [182]:
matched_dfs.loc[303128]['Title']

'Scale-Free Networks'

In [191]:
for i in sorted(np.argsort(tf_idf[45845].todense())[:5].tolist()[0][-10:], reverse = True):
    print(trigrams[i])

random graphs networks
random graphs
preferential attachment
networks social networks
networks degree distributions
networks degree
linking number
highly connected
graphs networks
degree distribution here


#### Woo! It makes sense! Trying on other papers

In [190]:
test_df = matched_dfs.sort_values('count', ascending = False).iloc[:300].sample(5)

for index, paper in test_df.iterrows():
    
    print('\n\n')
    print('Paper %s ' % paper['Title'])
    print('Key Phrases:\n')
    for i in sorted(np.argsort(tf_idf[index].todense())[:5].tolist()[0][-10:], reverse = True):
        print(trigrams[i])




Paper Generation, representation and flow of phase information in structure determination: recent developments in and around SHARP 2.0 
Key Phrases:

using sharp
sharp the
selenium sites
refined sharp
program sharp
phases calculated
initial phases calculated
initial phases
density modification
calculated sharp



Paper De Novo Gene Disruptions in Children on the Autistic Spectrum 
Key Phrases:

spectrum disorder
novo mutations
novo genes
nonsynonymous nonsynonymous
intellectual disability
de novo mutations
de novo genes
de novo
autism spectrum disorder
autism spectrum



Paper A short history of SHELX 
Key Phrases:

using shelxd
using shelx
se sites
positions determined
phases calculated
initial phases calculated
initial phases
heavy atom
found using
atom positions determined



Paper The high conductivity of iron and thermal evolution of the Earth’s core 
Key Phrases:

thermal convection
thermal conductivity
resistivity saturation
resistivity data
inner core
gomi et
electrical resi

In [189]:
matched_dfs.iloc[45845]['clean_text']

['Subjects were assigned a location in a network of 20 nodes, generated by a preferential attachment model; the network structure was created de novo for each session by attaching new nodes (each with two links) to existing nodes; and subjects were placed into the resulting networks at random.',
 'A soft thresholding power was chosen on the basis of the criterion of approximate scale-free topology.',
 'ÁngelesSerranoM\nMariánBoguñáM\nDmitriKrioukovD\nDepartment of Electrical Engineering, Computer Engineering and Informatics, Cyprus University of Technology, 33 Saripolou Street, 3036 Limassol, Cyprus\nCooperative Association for Internet Data Analysis (CAIDA), University of California, San Diego (UCSD), La Jolla, California 92093, USA\nDepartament de Física Fonamental, Universitat de Barcelona, Martí i Franquès 1, 08028 Barcelona, Spain\nf.papadopoulos@cut.ac.cy\ndima@ucsd.edu\nA framework is developed in which new connections to a growing network optimize geometric trade-offs between p